# Call Azure ML for PY Framework web service - Payroll Anomaly Detection

###  http://b03d85f2-b070-497e-a0f8-03ec2315c0af.westus.azurecontainer.io/score


### Imports

In [14]:
import json
import requests
from ngamlfpy.utils import pulled_json_to_df, pulled_df_to_json
from ngamlfpy.hrxmlconfig import MLModelConfig
from ngamlfpy.pipeline import FileFinder
import matplotlib.pyplot as plt
import pandas as pd
import os

ml_service = 'PAD'
num_emps_to_clip = 10



In [2]:
use_csv_input = True
model_code =  'A001' 
model_version = '001' 
#input_file_name =  'TWV_SS01_010_EUH_SOL_SUS_BIWK_PP9_100_U3_201915_input.csv'
input_file_name= 'PAD_A001_001_WDY_ALT_TST_ALTBM_WDT_000_sm_202003_input.csv'

finder = FileFinder(ml_service, use_model_name_in=True, use_model_name_out=False, model_name=model_code, model_version = model_version,
                         base_folder='./data', relative_input_folder=FileFinder.MLFOLDER_INPUT,
                         relative_output_folder=FileFinder.MLFOLDER_PREDICT)

full_path_in =  finder.get_full_input_file_name(input_file_name)  #os.path.join(args.data_folder, args.ml_service + '/input/' + args.model + '/' + args.model_version)

#full_path_in = os.path.join(folder,args.input_file_name)


df = pd.read_csv(full_path_in)
#df = pd.read_csv('data/PAD/input/' + model_code + '/PAD_EUH_AKN_A05_ALL_EP5_984_N0_201908_input.csv')
#df = pd.read_csv('data/PAD/input/M005/001/PAD_M005_001_EUH_ZZZ_Z10_MTHLY_EDF_310_ZZ_201905_input.csv')
display(df.head())

_,_,_,payroll_service, gcc, lcc, group, system, rest = finder.parse_input_file_name(input_file_name,include_remainder=True)
client, abkrs, period, other = rest.split('_')
ml_config = MLModelConfig.get_model_config_from_web_service_for_cust(ml_service, system=system,gcc=gcc,lcc=lcc,payroll_area=abkrs)


,Client,Employee_Id,Employee_Status,Employee_Type,Frequency,GCC,INPER,Id,Job_Code,LCC,...,WT_VIS,WT_VPY,WT_W2Box13PI [USA],WT_WA: Washington Paid Family & Medical Leave,WT_Withholding Order (Bankruptcy) [USA],WT_Withholding Order (Creditor) [USA],WT_Withholding Order (State Tax Levy) [USA],WT_Withholding Order (Student Loan) [USA],WT_Withholding Order (Support) [USA],WT_XPLC
0,0,7,Terminated,0,semi-monthly,ALT,202003,461176,0,TST,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,36,Terminated,0,semi-monthly,ALT,202003,529321,0,TST,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,75,Terminated,0,semi-monthly,ALT,202003,461275,0,TST,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,96,Terminated,0,semi-monthly,ALT,202003,461304,0,TST,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,162,Terminated,0,semi-monthly,ALT,202003,461408,0,TST,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


web service call successful:  api/model-info ?ml_service=PAD&system=WDT&gcc=ALT&lcc=TST&payroll_area=sm


In [33]:
save_csv_file_as_json = True

if use_csv_input:
    print('Using csv input')
    #j_predict = pulled_df_to_json(df,ml_config,'201902')
    j_predict = pulled_df_to_json(df,ml_config,period,use_first_data_line_as_selection=True,use_value_title_format=True,clip_emps = num_emps_to_clip)

    raw_data = {}
    raw_data['data'] = j_predict

    if save_csv_file_as_json:
       predict_json_file_name =  input_file_name.split('.')[0] + '.json'
       full_path_json = os.path.join(finder.get_output_folder(),predict_json_file_name)
       print('Writing json predict in file: ', full_path_json)
       with open(full_path_json, 'w') as outfile:
           json.dump(raw_data, outfile, indent=4)
           
            
else:        
    import pprint
    input_json_file_name = '' # <-- supply json file name here (including path)
    print ('Using Json predict file: ',input_json_file_name)

    with open(input_json_file_name) as json_data:
            j_predict = json.load(json_data)

print('Num emps: ',len(j_predict['values']))
print('')
print('First emp: ',j_predict['values'][0])
print('')
print('Selection: ',j_predict['selection'])
print('')     

Using csv input
Writing json predict in file:  ./data\PAD\predict\PAD_A001_001_WDY_ALT_TST_ALTBM_WDT_000_sm_202003_input.json
Num emps:  10

First emp:  [{'title': 'Client', 'value': 0}, {'title': 'Employee_Id', 'value': 7}, {'title': 'Employee_Status', 'value': 'Terminated'}, {'title': 'Employee_Type', 'value': '0'}, {'title': 'Frequency', 'value': 'semi-monthly'}, {'title': 'GCC', 'value': 'ALT'}, {'title': 'INPER', 'value': 202003}, {'title': 'Id', 'value': 461176}, {'title': 'Job_Code', 'value': 0}, {'title': 'LCC', 'value': 'TST'}, {'title': 'MLService', 'value': 'PAD'}, {'title': 'Model', 'value': 'A001'}, {'title': 'Pay_Code_1', 'value': 'Gross_Pay'}, {'title': 'Pay_Code_2', 'value': '0'}, {'title': 'Pay_Code_3', 'value': '0'}, {'title': 'Pay_Code_4', 'value': '0'}, {'title': 'Payinput_Type', 'value': '0'}, {'title': 'PayrollArea', 'value': 'sm'}, {'title': 'PayrollService', 'value': 'WDY'}, {'title': 'Period_Alight', 'value': 'semi-monthly|02/01/2020 - 02/15/2020|Chk1'}, {'titl

## Prepare headers

In [5]:
headers = {"Content-Type":"application/json"}
test_samples = json.dumps({'data':j_predict})
test_samples = bytes(test_samples, encoding='utf8')        
        
print (json.dumps(j_predict['selection'], sort_keys=False, indent=4, separators=(',', ': ')))

[
    {
        "title": "MLService",
        "value": "PAD"
    },
    {
        "title": "Model",
        "value": "A001"
    },
    {
        "title": "Version",
        "value": 1
    },
    {
        "title": "GCC",
        "value": "ALT"
    },
    {
        "title": "LCC",
        "value": "TST"
    },
    {
        "title": "PayrollService",
        "value": "WDY"
    },
    {
        "title": "System",
        "value": "WDT"
    },
    {
        "title": "Client",
        "value": 0
    }
]


# Call Prediction Web Service

### New  http://b03d85f2-b070-497e-a0f8-03ec2315c0af.westus.azurecontainer.io/score

- Predict Tax amounts per emp

### Imports

In [ ]:
import json
import requests
import pandas as pd
from ngamlfpy.utils import pulled_json_to_df, pulled_df_to_json
from ngamlfpy.hrxmlconfig import MLModelConfig
import matplotlib.pyplot as plt


In [ ]:


model_code =  'A001' #'M010'

if model_code == 'M010':
    model_version = '002'
    ml_service = 'PAD'
    payroll_service = 'EUH'
    gcc = 'AKN'
    lcc = 'A05'
    variant = 'ALL'
    system='EP5'
    client = '984'
    abkrs = 'N0'
    period = '201908'
elif model_code == 'M005':
    model_version = '001'
    ml_service = 'PAD'
    payroll_service = 'EUH'
    gcc = 'ZZZ'
    lcc = 'Z10'
    variant = 'MTHLY'
    system='EDF'
    client = '310'
    abkrs = 'ZZ'
    period = '201905'  
elif model_code == 'A001':
    model_version = '001'
    ml_service = 'PAD'
    payroll_service = 'WDY'
    gcc = 'ALT'
    lcc = 'TST'
    variant = 'ALTBM'
    system='WDT'
    client = '000'
    abkrs = 'sm'
    period = '202003'        
    
use_csv_input = True

save_csv_file_as_json = True

input_csv_file_name = 'data/' + ml_service + '/input/' + model_code + '/' + model_version + '/'  + '_'.join([ml_service,model_code,model_version,payroll_service,gcc,lcc,variant,system,client,abkrs,period]) + '_input.csv'
input_json_file_name = 'data/predict/' + model_code + '/' + model_code + '_test.json'
print(input_csv_file_name)
print(input_json_file_name)

# Convert csv file to json to use for predictions  if necessary

In [ ]:
df = pd.read_csv(input_csv_file_name)
#df = pd.read_csv('data/PAD/input/' + model_code + '/PAD_EUH_AKN_A05_ALL_EP5_984_N0_201908_input.csv')
#df = pd.read_csv('data/PAD/input/M005/001/PAD_M005_001_EUH_ZZZ_Z10_MTHLY_EDF_310_ZZ_201905_input.csv')
df.head()

In [ ]:
ml_config = MLModelConfig.get_model_config_from_web_service_for_cust(ml_service, system=system,gcc=gcc,lcc=lcc,payroll_area=abkrs)
#ml_config = MLModelConfig.get_model_config_from_web_service_for_cust('PAD', system='EP5',gcc='AKN',lcc='A05',payroll_area='N0')
#ml_config = MLModelConfig.get_model_config_from_web_service_for_cust('PAD', system='EDF',gcc='ZZZ',lcc='Z10',payroll_area='ZZ')

In [ ]:
if use_csv_input:
    print('Using csv input')
    j_predict = pulled_df_to_json(df,ml_config,period,use_first_data_line_as_selection=True,use_value_title_format=True)
    
    #Write json to disk for future use
    #outJson = 'data/predict/M010/M010_test.json'
    #outJson = 'data/predict/M005/M005_test.json'
    outJson = input_json_file_name
   
    
    if save_csv_file_as_json:
        print ('Writing out json file: ',outJson)
        with open(outJson, 'w') as outfile:
            json.dump(j_predict, outfile,indent=4)
            #json.dumps(json.loads(jTst), indent=4, sort_keys=True,outfile)
        
else:
    import pprint

    #model = 'M005'
    #predict_file_folder = model_code
    #predict_file_name = 'T001_test.json'
    #predict_file_name = 'T001_test_actually_with_T003_emp_data.json'
    #predict_file_name = 'T003_EQ1_402_ZCS_Z10_X3_201902_pulled.json'
    #predict_file_name = 'T001_EQ1_402_ZCS_Z10_X1_201902_pulled.json'
    #predict_file_name = model_code + '_test.json'
    #data_dir='data/predict'
    
    predict_source = 'json'
    #model_prefix = predict_file_folder 
    #predict_file_path = '/'.join([data_dir,model_prefix])
    #full_predict_file_name = '/'.join([predict_file_path,predict_file_name])
    
    print ('Using Json predict file: ',input_json_file_name)

    if predict_source == 'json':
        with open(input_json_file_name) as json_data:
            j_predict = json.load(json_data)

    # j_data_vals_stripped = []  
    # period = j_data['selection']['period']
    # for val in j_data['values']:
    #     if period == val['ForPeriod']:
    #         j_data_vals_stripped.append(val)
    # j_data['values']  = j_data_vals_stripped

print('Num emps: ',len(j_predict['values']))
print('')
print('First emp: ',j_predict['values'][0])
print('')
print('Selection: ',j_predict['selection'])
print('')    

### Prepare headers

In [ ]:
headers = {"Content-Type":"application/json"}
test_samples = json.dumps({'data':j_predict})
test_samples = bytes(test_samples, encoding='utf8')        

print (json.dumps(j_predict['selection'], sort_keys=False, indent=4, separators=(',', ': ')))

# Call Prediction Web Service
### Old - http://40.81.10.248:80/score
### Not quite as oldhttp://89c0489d-f162-4cb2-8a62-d551fc60a36c.westus.azurecontainer.io/score
### New  http://b03d85f2-b070-497e-a0f8-03ec2315c0af.westus.azurecontainer.io/score

- Predict Tax amounts per emp

In [6]:
response = requests.post("http://b03d85f2-b070-497e-a0f8-03ec2315c0af.westus.azurecontainer.io/score", test_samples, headers=headers)  #NGA_Tax_Withheld  Combined


if response.status_code == 200:
    json_predictions = response.json()
    print('Prediction Web Service call successful - ' + str(len(json_predictions['Predictions'])) + ' emps processed' + ' Model used: ' + json_predictions['info']['azure_model_name'] + ' Model config ws status: ' + json_predictions['info']['config_web_service_call_status'])

    print ('First 5 Tax predictions: ')
    for pred in json_predictions['Predictions'][:5]:
        print('   Anomaly Score: ' + str(pred))
else:
    print('web service failed. Status: ' + str(response.status_code) + ' message: ' +  str(response.text))

Prediction Web Service call successful - 10 emps processed Model used: PAD_model_A001_001 Model config ws status: Success
First 5 Tax predictions: 
   Anomaly Score: {'PERNR': '162', 'Score': -0.06432388465298289, 'Rank': 1, 'Important_Features': ['WT_HLTSAG', 'WT_SUI (ER) [USA]', 'WT_Total_ER_Taxes']}
   Anomaly Score: {'PERNR': '383', 'Score': -0.044786600459491344, 'Rank': 2, 'Important_Features': ['WT_SUI (ER) [USA]', 'WT_Total_ER_Taxes', 'WT_HLTSAG']}
   Anomaly Score: {'PERNR': '324', 'Score': -0.01935170283513421, 'Rank': 3, 'Important_Features': ['WT_Total_ER_Taxes', 'WT_SUI (ER) [USA]', 'WT_HLTSAG']}
   Anomaly Score: {'PERNR': '96', 'Score': -0.01510131852602814, 'Rank': 4, 'Important_Features': ['WT_HLTSAG', 'WT_Total_ER_Taxes', 'WT_SUI (ER) [USA]']}
   Anomaly Score: {'PERNR': '181', 'Score': -0.009525962357547102, 'Rank': 5, 'Important_Features': ['WT_HLTSAG', 'WT_Total_ER_Taxes', 'WT_SUI (ER) [USA]']}


In [7]:
json_predictions['info']

{'config_web_service_call_status': 'Success',
 'azure_model_name': 'PAD_model_A001_001',
 'model_code': 'A001',
 'model_version': '001',
 'pipeline_code_version': '0.1g',
 'predict_code_version': '0.1h'}

In [8]:
json_predictions['selection']

[{'title': 'MLService', 'value': 'PAD'},
 {'title': 'Model', 'value': 'A001'},
 {'title': 'Version', 'value': 1},
 {'title': 'GCC', 'value': 'ALT'},
 {'title': 'LCC', 'value': 'TST'},
 {'title': 'PayrollService', 'value': 'WDY'},
 {'title': 'System', 'value': 'WDT'},
 {'title': 'Client', 'value': 0}]

In [9]:
json_predictions

{'selection': [{'title': 'MLService', 'value': 'PAD'},
  {'title': 'Model', 'value': 'A001'},
  {'title': 'Version', 'value': 1},
  {'title': 'GCC', 'value': 'ALT'},
  {'title': 'LCC', 'value': 'TST'},
  {'title': 'PayrollService', 'value': 'WDY'},
  {'title': 'System', 'value': 'WDT'},
  {'title': 'Client', 'value': 0}],
 'info': {'config_web_service_call_status': 'Success',
  'azure_model_name': 'PAD_model_A001_001',
  'model_code': 'A001',
  'model_version': '001',
  'pipeline_code_version': '0.1g',
  'predict_code_version': '0.1h'},
 'Predictions': [{'PERNR': '162',
   'Score': -0.06432388465298289,
   'Rank': 1,
   'Important_Features': ['WT_HLTSAG',
    'WT_SUI (ER) [USA]',
    'WT_Total_ER_Taxes']},
  {'PERNR': '383',
   'Score': -0.044786600459491344,
   'Rank': 2,
   'Important_Features': ['WT_SUI (ER) [USA]',
    'WT_Total_ER_Taxes',
    'WT_HLTSAG']},
  {'PERNR': '324',
   'Score': -0.01935170283513421,
   'Rank': 3,
   'Important_Features': ['WT_Total_ER_Taxes',
    'WT_SUI

In [21]:
scores = [x['Score'] for x in json_predictions['Predictions']]
scores = scores[:num_emps_to_clip]
scores[:2]

[-0.06432388465298289, -0.044786600459491344]

In [23]:
important_features = [x['Important_Features'] for x in json_predictions['Predictions']]
important_features = important_features[:num_emps_to_clip]
important_features[:2]

[['WT_HLTSAG', 'WT_SUI (ER) [USA]', 'WT_Total_ER_Taxes'],
 ['WT_SUI (ER) [USA]', 'WT_Total_ER_Taxes', 'WT_HLTSAG']]

In [26]:
df_clipped = df[:num_emps_to_clip].copy()

In [27]:
df_clipped['Score'] = scores
df_clipped.head()

,Client,Employee_Id,Employee_Status,Employee_Type,Frequency,GCC,INPER,Id,Job_Code,LCC,...,WT_VPY,WT_W2Box13PI [USA],WT_WA: Washington Paid Family & Medical Leave,WT_Withholding Order (Bankruptcy) [USA],WT_Withholding Order (Creditor) [USA],WT_Withholding Order (State Tax Levy) [USA],WT_Withholding Order (Student Loan) [USA],WT_Withholding Order (Support) [USA],WT_XPLC,Score
0,0,7,Terminated,0,semi-monthly,ALT,202003,461176,0,TST,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.064324
1,0,36,Terminated,0,semi-monthly,ALT,202003,529321,0,TST,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.044787
2,0,75,Terminated,0,semi-monthly,ALT,202003,461275,0,TST,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.019352
3,0,96,Terminated,0,semi-monthly,ALT,202003,461304,0,TST,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.015101
4,0,162,Terminated,0,semi-monthly,ALT,202003,461408,0,TST,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.009526


In [30]:
df_features = pd.DataFrame(important_features,columns=['LF1','LF2','LF3'])
df_clipped = df_clipped.join(df_features)
df_clipped.head()

,Client,Employee_Id,Employee_Status,Employee_Type,Frequency,GCC,INPER,Id,Job_Code,LCC,...,WT_Withholding Order (Bankruptcy) [USA],WT_Withholding Order (Creditor) [USA],WT_Withholding Order (State Tax Levy) [USA],WT_Withholding Order (Student Loan) [USA],WT_Withholding Order (Support) [USA],WT_XPLC,Score,LF1,LF2,LF3
0,0,7,Terminated,0,semi-monthly,ALT,202003,461176,0,TST,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.064324,WT_HLTSAG,WT_SUI (ER) [USA],WT_Total_ER_Taxes
1,0,36,Terminated,0,semi-monthly,ALT,202003,529321,0,TST,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.044787,WT_SUI (ER) [USA],WT_Total_ER_Taxes,WT_HLTSAG
2,0,75,Terminated,0,semi-monthly,ALT,202003,461275,0,TST,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.019352,WT_Total_ER_Taxes,WT_SUI (ER) [USA],WT_HLTSAG
3,0,96,Terminated,0,semi-monthly,ALT,202003,461304,0,TST,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.015101,WT_HLTSAG,WT_Total_ER_Taxes,WT_SUI (ER) [USA]
4,0,162,Terminated,0,semi-monthly,ALT,202003,461408,0,TST,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.009526,WT_HLTSAG,WT_Total_ER_Taxes,WT_SUI (ER) [USA]


In [31]:
df_clipped_sorted = df_clipped.sort_values(by=['Score'])
df_clipped_sorted.head()

,Client,Employee_Id,Employee_Status,Employee_Type,Frequency,GCC,INPER,Id,Job_Code,LCC,...,WT_Withholding Order (Bankruptcy) [USA],WT_Withholding Order (Creditor) [USA],WT_Withholding Order (State Tax Levy) [USA],WT_Withholding Order (Student Loan) [USA],WT_Withholding Order (Support) [USA],WT_XPLC,Score,LF1,LF2,LF3
0,0,7,Terminated,0,semi-monthly,ALT,202003,461176,0,TST,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.064324,WT_HLTSAG,WT_SUI (ER) [USA],WT_Total_ER_Taxes
1,0,36,Terminated,0,semi-monthly,ALT,202003,529321,0,TST,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.044787,WT_SUI (ER) [USA],WT_Total_ER_Taxes,WT_HLTSAG
2,0,75,Terminated,0,semi-monthly,ALT,202003,461275,0,TST,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.019352,WT_Total_ER_Taxes,WT_SUI (ER) [USA],WT_HLTSAG
3,0,96,Terminated,0,semi-monthly,ALT,202003,461304,0,TST,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.015101,WT_HLTSAG,WT_Total_ER_Taxes,WT_SUI (ER) [USA]
4,0,162,Terminated,0,semi-monthly,ALT,202003,461408,0,TST,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.009526,WT_HLTSAG,WT_Total_ER_Taxes,WT_SUI (ER) [USA]
